In [3]:
# Install required packages (uncomment if needed)
!pip install opencv-python-headless easyocr matplotlib

import cv2
import easyocr
import numpy as np
import time

# Create EasyOCR reader once, outside the function for speed
reader = easyocr.Reader(['en'])

def recognize_plate(image_path, reader=reader):
    # Start timer
    start_time = time.time()

    # Load image
    img = cv2.imread(image_path)
    if img is None:
        print("Error: Image not found or invalid format.")
        return

    # Resize large images for speed (optional)
    if img.shape[1] > 800:
        scale = 800 / img.shape[1]
        img = cv2.resize(img, (800, int(img.shape[0]*scale)))

    # Pre-processing
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur = cv2.bilateralFilter(gray, 11, 17, 17)
    edges = cv2.Canny(blur, 30, 200)

    # Find contours
    contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    possible_plates = []

    # Filter plate candidates by aspect ratio/size
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        if w > 80 and h > 20 and w/h < 6 and w/h > 2:
            possible_plates.append((x, y, w, h))

    found = False
    for x, y, w, h in possible_plates:
        plate_img = gray[y:y + h, x:x + w]
        result = reader.readtext(plate_img)
        if result:
            print("Detected License Plate Text:", result[0][1])
            found = True
            break

    if not found:
        print("No license plate detected.")

    # End timer and report time
    end_time = time.time()
    print(f"Total processing time: {end_time - start_time:.3f} seconds")

# Usage Example (replace 'your_image_here.jpg' with your own image filename)
recognize_plate('/content/Cars82.png')

Detected License Plate Text: UP46 IClJeo
Total processing time: 0.278 seconds


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
